<div class="alert alert-block alert-info"><h1>统计学习方法课后作业</h1></div>
<div class="alert alert-block alert-success"> <h3>作者：学号:Z21160043 姓名:刘奥</h3></div> 

Read data from file 'simple.csv'. One line is a transaction. Get a list of all transactions.

Hint: Dimensionality reduction
<blockquote>
from itertools import chain<br>
one = list(chain.from_iterable(data))
</blockquote>

In [1]:
from pathlib import Path
f = Path('simple.csv')
print(f.read_text())

Bread, Butter
Bread, Jelly
Bread, Milk, Butter
Chips, Milk
Bread, Chips
Bread, Jelly, Peanut, Butter
Bread, Milk
Chips, Jelly


In [2]:
data = [set(line.split(',')) for line in f.read_text().replace(' ','').split('\n')]
data

[{'Bread', 'Butter'},
 {'Bread', 'Jelly'},
 {'Bread', 'Butter', 'Milk'},
 {'Chips', 'Milk'},
 {'Bread', 'Chips'},
 {'Bread', 'Butter', 'Jelly', 'Peanut'},
 {'Bread', 'Milk'},
 {'Chips', 'Jelly'}]

In [3]:
#set n as the number of dataset
n=len(data)
n

8

List all unique items

In [4]:
from itertools import chain
items = set(chain.from_iterable(data))
items

{'Bread', 'Butter', 'Chips', 'Jelly', 'Milk', 'Peanut'}

Calculate the number of one itemset

In [5]:
def number(itemset):
    return sum([itemset.issubset(tran) for tran in data])

number({'Bread', 'Butter'})

3

Calculate the support of one itemset

In [6]:
def support(itemset): 
    return number(itemset)/n

support({'Bread', 'Butter'})

0.375

Calculate the confidence of one itemset

In [7]:
def confidence(itemset1, itemset2):
    return support(itemset1|itemset2)/support(itemset1)

confidence({'Bread'}, {'Butter'})

0.5

Calculate the number of all 1-itemset

In [8]:
itemset_num_1 = [{item:number({item})} for item in items]
print(itemset_num_1)

[{'Butter': 3}, {'Bread': 6}, {'Jelly': 3}, {'Peanut': 1}, {'Chips': 3}, {'Milk': 3}]


Calculate the support of all 1-itemset

In [9]:
itemset_sup_1 = [{item:support({item})} for item in items]
print(itemset_sup_1)

[{'Butter': 0.375}, {'Bread': 0.75}, {'Jelly': 0.375}, {'Peanut': 0.125}, {'Chips': 0.375}, {'Milk': 0.375}]


Calculate the confidence of all 1-itemset

In [10]:
item_con_1 = [{f'{item1}->{item2}': confidence({item1}, {item2})} for item1 in items for item2 in items if item1 != item2]
item_con_1[:5]

[{'Butter->Bread': 1.0},
 {'Butter->Jelly': 0.3333333333333333},
 {'Butter->Peanut': 0.3333333333333333},
 {'Butter->Chips': 0.0},
 {'Butter->Milk': 0.3333333333333333}]

In [11]:
C1 = [frozenset({item}) for item in items]
C1

[frozenset({'Butter'}),
 frozenset({'Bread'}),
 frozenset({'Jelly'}),
 frozenset({'Peanut'}),
 frozenset({'Chips'}),
 frozenset({'Milk'})]

Using Apriori Algorithm to calculate the frequent itemset of the data in file 'simple.csv'

In [12]:
sigma = 0.2
C1 = [frozenset({item}) for item in items]
Lk = [[i for i in C1 if support(i)>sigma]]
while Lk[-1]!=[]: # L[-1] The last item
    Ck = set([ i|j for i in  Lk[-1] for j in Lk[-1] if i != j ])
    Lk .append([i for i in Ck if support(i)>sigma])
Lk = Lk[:-1]
Lk

[[frozenset({'Butter'}),
  frozenset({'Bread'}),
  frozenset({'Jelly'}),
  frozenset({'Chips'}),
  frozenset({'Milk'})],
 [frozenset({'Bread', 'Milk'}),
  frozenset({'Bread', 'Butter'}),
  frozenset({'Bread', 'Jelly'})]]

In [13]:
# Another complete way
class Aproiri():
    def __init__(self,data,sigma,phi):
        self.sigma = sigma
        self.phi = phi
        self.data = data
    
    def Create_C1(self):
        items = set(chain.from_iterable(self.data))
        C1 = [frozenset({item}) for item in items]
        return C1
    
    def Cal_Lk(self, Ck):
        Lk = [frozenset(itemset) for itemset in Ck if support(itemset) > self.sigma]
        Lk_support = [{itemset:support(itemset)} for itemset in Lk]
        return Lk, Lk_support
    
    def Creat_C_kadd1(self,Lk):
        Ck = []
        for i in range(len(Lk)):
            for j in range(i+1,len(Lk)):
                if len(Lk[i]-Lk[j])==1:
                    if (Lk[i]|Lk[j]) not in Ck:
                        Ck.append(frozenset(Lk[i]|Lk[j]))
        return Ck
    
    def apriori(self):
        C1 = self.Create_C1()
        L1, L1_support = self.Cal_Lk(C1)
        k = 1
        L_union = []
        L_union_support = []
        L_union.extend(L1)
        L_union_support.extend(L1_support)
        while k <= n:
            C_kadd1= self.Creat_C_kadd1(L1)
            Lk,Lk_support = self.Cal_Lk(C_kadd1)
            if Lk == []:
                break
            L_union.extend(Lk)
            L_union_support.extend(Lk_support)
            k+=1
            L1 = Lk
        return L_union, L_union_support
    
    def association_rules(self, L_union, L_union_support):
        rules = []
        length = len(L_union)
        for i in range(length):
            for j in range(i+1,length):
                    if L_union[i] < L_union[j]:
                        support = L_union_support[j][L_union[j]]
                        conf = confidence(L_union[i],L_union[j])
                        if conf >= self.phi:
                            rule = ({f'{L_union[i]}-->{L_union[j] - L_union[i]}(support:{support})': conf})
                            rules.append(rule)
        return rules

In [14]:
apro = Aproiri(data,0.2,0.5)
L_union, L_union_support = apro.apriori()
L_union_support

[{frozenset({'Butter'}): 0.375},
 {frozenset({'Bread'}): 0.75},
 {frozenset({'Jelly'}): 0.375},
 {frozenset({'Chips'}): 0.375},
 {frozenset({'Milk'}): 0.375},
 {frozenset({'Bread', 'Butter'}): 0.375},
 {frozenset({'Bread', 'Jelly'}): 0.25},
 {frozenset({'Bread', 'Milk'}): 0.25}]

In [15]:
rules = apro.association_rules(L_union,L_union_support)
rules

[{"frozenset({'Butter'})-->frozenset({'Bread'})(support:0.375)": 1.0},
 {"frozenset({'Bread'})-->frozenset({'Butter'})(support:0.375)": 0.5},
 {"frozenset({'Jelly'})-->frozenset({'Bread'})(support:0.25)": 0.6666666666666666},
 {"frozenset({'Milk'})-->frozenset({'Bread'})(support:0.25)": 0.6666666666666666}]

Using Apriori Algorithm to calculate the frequent itemset of the data in file 'Market_Basket_Optimisation.csv'

In [16]:
%%time
# New data sets redefine variable functions
f_Mar = Path('Market_Basket_Optimisation.csv')
data = [set(line.split(',')) for line in f_Mar.read_text().strip().split('\n')]
n=len(data)
def number(itemset):
    return sum([itemset.issubset(tran) for tran in data])
def support(itemset): 
    return number(itemset)/n
def confidence(itemset1, itemset2):
    return support(itemset1|itemset2)/support(itemset1)
items = set(chain.from_iterable(data))

CPU times: user 26.1 ms, sys: 9.71 ms, total: 35.8 ms
Wall time: 33.3 ms


In [19]:
sigma = 0.05
C1 = [frozenset({item}) for item in items]
Lk = [[i for i in C1 if support(i)>sigma]]
while Lk[-1]!=[]:
    Ck = set([ i|j for i in  Lk[-1] for j in Lk[-1] if i != j ])
    Lk .append([i for i in Ck if support(i)>sigma])
Lk = Lk[:-1]
Lk

[[frozenset({'pancakes'}),
  frozenset({'tomatoes'}),
  frozenset({'ground beef'}),
  frozenset({'mineral water'}),
  frozenset({'spaghetti'}),
  frozenset({'low fat yogurt'}),
  frozenset({'olive oil'}),
  frozenset({'frozen smoothie'}),
  frozenset({'eggs'}),
  frozenset({'chicken'}),
  frozenset({'escalope'}),
  frozenset({'cooking oil'}),
  frozenset({'whole wheat rice'}),
  frozenset({'green tea'}),
  frozenset({'turkey'}),
  frozenset({'chocolate'}),
  frozenset({'french fries'}),
  frozenset({'milk'}),
  frozenset({'cake'}),
  frozenset({'cookies'}),
  frozenset({'shrimp'}),
  frozenset({'soup'}),
  frozenset({'grated cheese'}),
  frozenset({'frozen vegetables'}),
  frozenset({'burgers'})],
 [frozenset({'mineral water', 'spaghetti'}),
  frozenset({'eggs', 'mineral water'}),
  frozenset({'chocolate', 'mineral water'})]]

In [20]:
apro_mar = Aproiri(data,0.05,0.05)
L_union, L_union_support = apro_mar.apriori()
L_union_support[:5] # print  before five 

[{frozenset({'pancakes'}): 0.09505399280095987},
 {frozenset({'tomatoes'}): 0.06839088121583789},
 {frozenset({'ground beef'}): 0.09825356619117451},
 {frozenset({'mineral water'}): 0.23836821757099053},
 {frozenset({'spaghetti'}): 0.17411011865084655}]

In [21]:
rules = apro_mar.association_rules(L_union,L_union_support)
rules

[{"frozenset({'mineral water'})-->frozenset({'spaghetti'})(support:0.05972536995067324)": 0.2505592841163311},
 {"frozenset({'mineral water'})-->frozenset({'eggs'})(support:0.05092654312758299)": 0.21364653243847875},
 {"frozenset({'mineral water'})-->frozenset({'chocolate'})(support:0.05265964538061592)": 0.220917225950783},
 {"frozenset({'spaghetti'})-->frozenset({'mineral water'})(support:0.05972536995067324)": 0.3430321592649311},
 {"frozenset({'eggs'})-->frozenset({'mineral water'})(support:0.05092654312758299)": 0.28338278931750743},
 {"frozenset({'chocolate'})-->frozenset({'mineral water'})(support:0.05265964538061592)": 0.3213995117982099}]

Using FP Growth Algorithm to calculate the frequent itemset of the data in file 'FPGrowth.csv'

In [22]:
%%time
from pathlib import Path
f_fp = Path('FPGrowth.csv')
dataSet = [list(line.split(',')) for line in f_fp.read_text().replace(' ','').split('\n')]
# with open('FPGrowth.csv', 'r') as f:
#     data = [list(map(lambda item: item.strip(), trans.split(','))) for trans in f] # use map + lambda
# data
dataSet

CPU times: user 475 µs, sys: 1.04 ms, total: 1.51 ms
Wall time: 1.23 ms


[['I1', 'I2', 'I5'],
 ['I2', 'I4'],
 ['I2', 'I3', 'I6'],
 ['I1', 'I2', 'I4'],
 ['I1', 'I3'],
 ['I2', 'I3'],
 ['I1', 'I3'],
 ['I1', 'I2', 'I3', 'I5'],
 ['I1', 'I2', 'I3']]

In [23]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count) # The blank space * ind
        for child in self.children.values():
            child.disp(ind+1)
rootNode = treeNode('pyramid',9,None)
rootNode.children['eye'] = treeNode('eye',13,None)
rootNode.disp()

   pyramid   9
     eye   13


In [24]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
#     print('freqItemSet: ',freqItemSet)
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
#     print('headerTable: ',headerTable)
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)] # sorted by big to small
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

In [25]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [26]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [27]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        if frozenset(trans) in retDict:
            retDict[frozenset(trans)] += 1
        else:
            retDict[frozenset(trans)] = 1
    return retDict
initSet = createInitSet(dataSet)
initSet

{frozenset({'I1', 'I2', 'I5'}): 1,
 frozenset({'I2', 'I4'}): 1,
 frozenset({'I2', 'I3', 'I6'}): 1,
 frozenset({'I1', 'I2', 'I4'}): 1,
 frozenset({'I1', 'I3'}): 2,
 frozenset({'I2', 'I3'}): 1,
 frozenset({'I1', 'I2', 'I3', 'I5'}): 1,
 frozenset({'I1', 'I2', 'I3'}): 1}

In [28]:
myFPtree, myHeaderTab = createTree(initSet, 2)
myFPtree.disp()
myHeaderTab

   Null Set   1
     I2   7
       I1   4
         I5   1
         I4   1
         I3   2
           I5   1
       I4   1
       I3   2
     I3   2
       I1   2


{'I5': [2, <__main__.treeNode at 0x7f3ed4739820>],
 'I2': [7, <__main__.treeNode at 0x7f3ed4739b20>],
 'I1': [6, <__main__.treeNode at 0x7f3ed4739a60>],
 'I4': [2, <__main__.treeNode at 0x7f3ed4739af0>],
 'I3': [6, <__main__.treeNode at 0x7f3ed4739160>]}

In [29]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [30]:
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats
findPrefixPath('I1', myHeaderTab['I1'][1])

{frozenset({'I2'}): 4, frozenset({'I3'}): 2}

In [31]:
def mineFPtree(inTree, headerTable, minSup, preFix, freqItemList):
    orderedItems = [v[0] for v in sorted(headerTable.items(), key=lambda p:p[1][0])] # big to small
    for item in orderedItems: 
        newFreqSet = preFix.copy() # The next level must be the same as the previous level --requires copy
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(item, headerTable[item][1]) # Conditional pattern base
        myCondTree, myConHead = createTree(condPattBases, minSup)  # Conditional FP Tree
        if myConHead != None:
            mineFPtree(myCondTree, myConHead, minSup, newFreqSet, freqItemList) # Recursive mining
freqItemList = []          
mineFPtree(myFPtree, myHeaderTab, 2, set([]), freqItemList)
freqItemList

[{'I5'},
 {'I2', 'I5'},
 {'I1', 'I5'},
 {'I1', 'I2', 'I5'},
 {'I4'},
 {'I2', 'I4'},
 {'I1'},
 {'I1', 'I3'},
 {'I1', 'I2'},
 {'I3'},
 {'I1', 'I3'},
 {'I1', 'I2', 'I3'},
 {'I2', 'I3'},
 {'I2'}]

Using FP Growth Algorithm to calculate the frequent itemset of the data in file 'Market_Basket_Optimisation.csv'

In [34]:
fMar = Path('Market_Basket_Optimisation.csv')
dataSet = [list(line.split(',')) for line in fMar.read_text().strip().split('\n')]
# print(dataSet[:5])
initSet = createInitSet(dataSet)
myFPtree, myHeaderTab = createTree(initSet, 50)
freqItemList = []          
mineFPtree(myFPtree, myHeaderTab, 2, set([]), freqItemList)
freqItemList[:10] # 50 freq / before 10

[{'gluten free bar'},
 {'extra dark chocolate', 'gluten free bar'},
 {'extra dark chocolate', 'gluten free bar', 'mineral water'},
 {'gluten free bar', 'mushroom cream sauce'},
 {'gluten free bar', 'mushroom cream sauce', 'salmon'},
 {'gluten free bar', 'honey', 'mushroom cream sauce'},
 {'gluten free bar', 'honey', 'mushroom cream sauce', 'salmon'},
 {'frozen vegetables', 'gluten free bar', 'mushroom cream sauce'},
 {'frozen vegetables', 'gluten free bar', 'mushroom cream sauce', 'salmon'},
 {'frozen vegetables', 'gluten free bar', 'honey', 'mushroom cream sauce'}]